In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
	temperature=0.1, 
	streaming=True,
	callbacks=[
		StreamingStdOutCallbackHandler()
	]
)

template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}?",)
prompt = template.format(country_a="Mexico", country_b="Thailand")
chat.predict(prompt)

In [ ]:
template2 = ChatPromptTemplate.from_messages(
	[
		("system", "You are a geography expert. And you only reply in {language}."),
		("ai", "Ciao, mi chiamo {name}!"),
		("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?"),
	]
)

prompt2 = template2.format_messages(
	language="Greek",
	name="Socrates",
	country_a="Mexico",
	country_b="Thailand",
)

prompt2
chat.predict_messages(prompt2)

In [ ]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
	
	def parse(self, text):
		items = text.strip().split(",")
		return list(map(str.strip, items))


p = CommaOutputParser()
p.parse("Hello, how, are, you?")

In [ ]:
# bad request

template3 = ChatPromptTemplate.from_messages(
	[
		(
			"system", 
			"You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do NOT reply with anything else."
		),
		(
			"human",
			"{question}",
		),
	]
)

prompt3 = template3.format_messages(
	max_items=10,
	question="What are the colors?",
)

result = chat.predict_messages(prompt3)

p = CommaOutputParser()
p.parse(result.content)

In [ ]:
# good request with chain

chain = template3 | chat | CommaOutputParser()
chain.invoke({
	"max_items": 5,
	"question": "what are the pokemons?"
})

In [ ]:
chef_prompt = ChatPromptTemplate.from_messages([
	(
		"system",
		"You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."
	),
	(
		"human",
		"I want to cook {cuisine} food." 
	),
])

chef_chain = chef_prompt | chat


veg_chef_prompt = ChatPromptTemplate.from_messages([
	(
		"system",
		"You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain thier preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."
	),
	(
		"human",
		"{recipe}" 
	),
])

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe" : chef_chain} | veg_chain

final_chain.invoke({
	"cuisine" : "indian",
})